# 1. Gen inventory.csv

In [1]:
import pandas as pd
import random

# Đọc dữ liệu gốc
df = pd.read_csv("fragrance_collection.csv")

# Lấy ngẫu nhiên 300 dòng
inventory = df[['Brand', 'Name']].sample(n=300, random_state=42).reset_index(drop=True)

# Thêm cột Quantity với giá trị ngẫu nhiên từ 1 đến 20
inventory['Quantity'] = [random.randint(1, 20) for _ in range(300)]

In [2]:
inventory.head(10)

,Brand,Name,Quantity
0,Jovan,Jovan Black Musk,9
1,Amouage,Amouage Sunshine,17
2,Dunhill,Dunhill Desire Blue Ocean,9
3,Batalat,Batalat Clementine Lmprober Oud,12
4,Giorgio Armani,Giorgio Armani My Way Gift Set,8
5,Kun Safi,Kun Safi Honey Safi Perfume Oil,7
6,Bvlgari,Bvlgari Omnia Crystalline Gift Set,9
7,Salvatore Ferragamo,Salvatore Ferragamo Signorina In Fiore,3
8,B96,B96 Michel Girard Geranium Pepper EDP,2
9,Joyau Unique & Sensoriel Parfums,Joyau Unique & Sensoriel Parfums Coffeeze EDP,18


# 2. Convert SAR to VND

In [7]:
df = pd.read_csv("fragrance_collection.csv")

# Price SAR -> VND
df['Price'] = df['Price'] * 6900

In [8]:
df.to_csv("fragrance_collection_vnd.csv", index=False)

# 3. FAISS VectorDB

In [1]:
import pandas as pd
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [3]:
df = pd.read_csv("fragrance_collection_vnd.csv")

def row_to_document(row):
    text = " | ".join([f"{col}: {str(row[col])}" for col in row.index if col not in ['Price', 'image', 'Rate', 'Rating_count', 'Size', 'Year']])
    return Document(page_content=text)

documents = [row_to_document(row) for _, row in df.iterrows()]

documents[:5]

[Document(page_content="Name: Dolce & Gabanna L'imperatrice 3 Pour Femme | Description: Perfume for the energetic woman who is a hero in her movie in life every day! It keeps you vibrant and sparkling with irresistible attractiveness, a combination of kiwi, lemon and sandalwood. | Brand: Dolce&Gabbana | Gender: Women | Product_Type: Perfume | Character_x: Romantic | Fragrance_Family: Floral | Ingredients: watermerlon, kiwi, pink cyclamen, musk, pink pepper, jasmine, sandalwood, lemon tree | Concentration: Eau de Toilette | Top_note: pink pepper, kiwi, rhubarb | Middle_note: jasmine, cyclamen, watermelon | Base_note: musk, sandalwood, lemon trees."),
 Document(page_content='Name: Roberto Cavalli Paradiso | Description: Woody floral fragrance, a subtle aroma that makes you feel fresh with a stunning blend of citrus, jasmine, and cypress. | Brand: Roberto Cavalli | Gender: Women | Product_Type: Perfume | Character_x: Romantic | Fragrance_Family: Woody | Ingredients: citrus, mandarin, berg

In [4]:
# HuggingFace Embedding
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# FAISS index
db = FAISS.from_documents(documents, embedding_model)

# Save FAISS index
db.save_local("faiss_index")

In [5]:
# Usage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Load FAISS index from folder
db = FAISS.load_local("faiss_index", 
                      embedding_model,
                      allow_dangerous_deserialization=True)

In [ ]:
!pip install -q deep-translator

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator()

# Similarity search
query = "chai nước hoa nữ"
query = GoogleTranslator(source='vi', target='en').translate(query)
print(query)

results = db.similarity_search(query, k=100)

# Print results
for doc in results:
    print(doc.page_content)

Women's perfume bottle
Name: Burberry Her Perfume Set  | Description: An exquisite perfume set for women that contains a 100 ml EDP, in addition to a 7.5 ml bottle of the same perfume and a 75 ml perfumed body lotion.
The set comes to you in a distinctive and elegant package, which you could easily gift to your beloved ones on happy and special occasions.
The scent of the perfume embodies the young and lively spirit of the charming city of London, the creative city with its ancient pillars.
The floral heart of the scent highlights the subtle details of your delicate femininity and highlights your beauty. | Brand: Burberry | Gender: Women | Product_Type: Perfume Set | Character_x: Romantic | Fragrance_Family: Floral | Ingredients: lemon, mandarin, black currant, sour cherry, blackberry, raspberry, strawberry, jasmine, violet, patchouli, amber, woods, cashmeran, oak moss, vanilla, musk | Concentration: Eau de Parfum | Top_note: lemon, mandarin, black currant, sour cherry, blackberry, ras

# 4. Build SQLite

In [33]:
import pandas as pd
import sqlite3

# Load CSV files
fragrance_df = pd.read_csv("fragrance_collection_vnd.csv")
inventory_df = pd.read_csv("inventory.csv")

# Tạo connection tới SQLite DB (sẽ tạo file nếu chưa có)
conn = sqlite3.connect("perfumes.db")

# Đưa dữ liệu vào các bảng SQLite
fragrance_df.to_sql("fragrance_collection", conn, if_exists="replace", index=False)
inventory_df.to_sql("inventory", conn, if_exists="replace", index=False)

# Đóng connection
conn.close()

In [34]:
import sqlite3

conn = sqlite3.connect("perfumes.db")
cursor = conn.cursor()

# Kiểm tra 5 dòng đầu trong bảng
cursor.execute("SELECT * FROM fragrance_collection LIMIT 5")
print(cursor.fetchall())

cursor.execute("SELECT * FROM inventory LIMIT 5")
print(cursor.fetchall())

conn.close()


[("Dolce & Gabanna L'imperatrice 3 Pour Femme", 1373100, 'Perfume for the energetic woman who is a hero in her movie in life every day! It keeps you vibrant and sparkling with irresistible attractiveness, a combination of kiwi, lemon and sandalwood.', '5', '6', 'https://assets.goldenscent.com/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/3/4/3423473020615-dolce-_-gabbana-l_imperatrice-01_1_.png', 'Dolce&Gabbana', 'Women', 'Perfume', 'Romantic', 'Floral', '100 ml', 2009, 'watermerlon, kiwi, pink cyclamen, musk, pink pepper, jasmine, sandalwood, lemon tree', 'Eau de Toilette', 'pink pepper, kiwi, rhubarb', 'jasmine, cyclamen, watermelon', 'musk, sandalwood, lemon trees.'), ('Roberto Cavalli Paradiso', 1166100, 'Woody floral fragrance, a subtle aroma that makes you feel fresh with a stunning blend of citrus, jasmine, and cypress.', '4.95', '17', 'https://assets.goldenscent.com/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/3/6/3607347733423c-roberto-cavall

In [37]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load model local
model_id = "phatvucoder/Qwen2.5-1.5B-Perfumassist"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=pipe)

# Tạo database và agent như trên
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent

db = SQLDatabase.from_uri("sqlite:///perfumes.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)
agent.run("Loại nước hoa nào có tone hương vani?")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use mps:0
/var/folders/h3/4_pq_7pd3pq2286xkp78c1480000gn/T/ipykernel_26157/196035688.py:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=pipe)
/var/folders/h3/4_pq_7pd3pq2286xkp78c1480000gn/T/ipykernel_26157/196035688.py:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  agent.run("Loại nước hoa nào có tone hương vani?")




> Entering new SQL Agent Executor chain...


/Users/phatvu/miniconda3/lib/python3.11/site-packages/transformers/generation/utils.py:2208: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(
Exception in thread Thread-11 (generate):
Traceback (most recent call last):
  File "/Users/phatvu/miniconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/phatvu/.local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/Users/phatvu/miniconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/phatvu/miniconda3/lib/python3.11/si

Empty: 

In [39]:
!pip install -q langchain_huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chat-with-mlx 0.2.2 requires chromadb==0.5.0, which is not installed.
langchain-chroma 0.1.2 requires chromadb<0.6.0,>=0.4.0, which is not installed.
chat-with-mlx 0.2.2 requires langchain==0.1.16, but you have langchain 0.2.15 which is incompatible.
chat-with-mlx 0.2.2 requires langchain-core==0.1.45, but you have langchain-core 0.3.49 which is incompatible.
chat-with-mlx 0.2.2 requires pypdf==4.2.0, but you have pypdf 4.3.1 which is incompatible.
langchain-chroma 0.1.2 requires langchain-core<0.3,>=0.1.40, but you have langchain-core 0.3.49 which is incompatible.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.28.1 which is incompatible.
langchain 0.2.15 requires langchain-core<0.3.0,>=0.2.35, but you have langchain-core 0.3.49 which is incompatible.
langchain 0.2.15 requires langsmith<0.2.0,>=

In [41]:
import torch
import warnings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent

# Ignore deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Ép model về CPU
device = torch.device("cpu")
model_id = "phatvucoder/Qwen2.5-1.5B-Perfumassist"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device=-1  # use CPU
)

llm = HuggingFacePipeline(pipeline=pipe)

# Kết nối DB và tạo agent
db = SQLDatabase.from_uri("sqlite:///perfumes.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

# Dùng .invoke thay vì .run (theo cảnh báo mới)
response = agent.invoke("Loại nước hoa nào có hương chính là hoa nhài?")
print(response)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: 
Observation: ['fragrance_notes', 'fragrance_collection, inventory

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: ` From the above observations, I need to find out which table(s) may contain the names or details`